# 15 · Gravitational Lensing Demo (SpectraMind V50)

Educational, **mission‑grade** demo that shows how a simple microlensing event can bias *observed* transit data products when sampling/weighting effects are present. This notebook is **pipeline‑safe**:

- It **does not** implement pipeline logic; it *reads* artifacts produced by the CLI (if present) or synthesizes a clean spectrum.
- It simulates a **point‑lens microlensing** light curve during transit and demonstrates how temporal magnification + wavelength‑dependent weights can create apparent spectral biases.
- Artifacts are written under `outputs/notebooks/15_gravitational_lensing/` for DVC tracking.

**What you'll do**
1. Load a base transmission spectrum (from `outputs/` or synthesize).
2. Simulate microlensing magnification `A(t)` with typical point‑lens formula.
3. Combine `A(t)` with a simple exposure model and wavelength‑dependent weights to build an **effective** spectrum.
4. Compare original vs lensed spectra; visualize Einstein‑curve, residuals, and band overlays.
5. Export a compact diagnostics bundle (JSON + CSV + PNGs).

In [ ]:
import os, sys, json, platform
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook'); sns.set_style('whitegrid')

ROOT = Path.cwd().resolve()
NB_OUT = ROOT / 'outputs' / 'notebooks' / '15_gravitational_lensing'
NB_OUT.mkdir(parents=True, exist_ok=True)

ENV = {'python': platform.python_version(), 'platform': platform.platform()}
(NB_OUT/'env_snapshot.json').write_text(json.dumps(ENV, indent=2))
print('ROOT:', ROOT) ; print('NB_OUT:', NB_OUT)

## 0) Background: point‑lens microlensing
For a point lens, the (achromatic) magnification for a projected separation `u = θ/θ_E` is:

\begin{equation}
A(u) = \frac{u^2 + 2}{u\,\sqrt{u^2 + 4}}\,.
\end{equation}

With constant proper motion, `u(t) = \sqrt{u_0^2 + ((t - t_0)/t_E)^2}`. In reality lensing is achromatic, but **effective** spectral biases can arise if magnification varies during a transit while the instrument/pipeline applies time‑/wavelength‑dependent weights (exposure timing, throughput, flagging). We illustrate this effect using a simplified toy model.

## 1) Load base spectrum
We attempt to load a predictions table from `outputs/` and select one spectrum; if not found, we synthesize a smooth spectrum with two absorption features.

In [ ]:
def find_candidates():
    roots = [ROOT/'outputs']
    pats = ['**/predictions.csv','**/mu.csv','**/spectra.npy','**/mu.npy']
    cands = []
    for r in roots:
        for pat in pats:
            cands += list(r.glob(pat))
    return sorted(set(cands), key=lambda p: p.stat().st_mtime)

def load_one_spectrum(path: Path) -> pd.DataFrame:
    if path.suffix=='.npy':
        arr = np.load(path)
        if arr.ndim==1: arr = arr[None,:]
        mu = arr[0]
        return pd.DataFrame({'wavelength_index': np.arange(len(mu)), 'mu': mu})
    df = pd.read_csv(path)
    cols = {c.lower(): c for c in df.columns}
    if {'planet_id','wavelength_index','mu'}.issubset(cols):
        one = df[df[cols['planet_id']]==df[cols['planet_id']].iloc[0]].copy()
        one = one.rename(columns={cols['wavelength_index']:'wavelength_index', cols['mu']:'mu'})
        return one[['wavelength_index','mu']].sort_values('wavelength_index').reset_index(drop=True)
    mu_cols = [c for c in df.columns if str(c).startswith('mu_')]
    if mu_cols:
        mu = df[mu_cols].iloc[0].to_numpy(float)
        return pd.DataFrame({'wavelength_index': np.arange(len(mu)), 'mu': mu})
    raise ValueError(f'Unsupported schema for {path}')

CANDS = find_candidates()
if not CANDS:
    # synthesize a clean spectrum (283 bins) with two features
    x = np.linspace(0, 1, 283)
    mu_clean = 0.01 + 0.002*np.exp(-0.5*((x-0.30)/0.06)**2) + 0.0015*np.exp(-0.5*((x-0.70)/0.05)**2)
    base = pd.DataFrame({'wavelength_index': np.arange(283), 'mu': mu_clean})
    print('No outputs found; using synthetic spectrum.')
else:
    base = load_one_spectrum(CANDS[-1])
    print('Loaded from:', CANDS[-1].relative_to(ROOT))
base.head()

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(base['wavelength_index'], base['mu'], lw=1.5)
plt.title('Base transmission spectrum (μ)')
plt.xlabel('wavelength index'); plt.ylabel('μ (arb)')
plt.tight_layout(); plt.savefig(NB_OUT/'base_spectrum.png', dpi=150); plt.close()
print('Saved base_spectrum.png')

## 2) Microlensing model in time
We simulate a simple transit time series with uniform exposure spacing and a point‑lens magnification that varies across the observing window.

**Parameters** (tweak to taste):
- `u0` : minimum impact parameter in Einstein‑radius units (smaller → stronger peak).
- `tE` : Einstein timescale half‑width (controls event duration).
- `t0` : time of closest approach (center of peak).

In [ ]:
def A_point_lens(u):
    u = np.asarray(u, float)
    return (u*u + 2) / (u * np.sqrt(u*u + 4))

def u_of_t(t, t0=0.0, u0=0.2, tE=0.3):
    return np.sqrt(u0*u0 + ((t - t0)/tE)**2)

# Exposure grid (normalized time)
N_EXP = 200
t = np.linspace(-0.8, 0.8, N_EXP)
A = A_point_lens(u_of_t(t, t0=0.0, u0=0.20, tE=0.25))

plt.figure(figsize=(8,3))
plt.plot(t, A, lw=1.6)
plt.title('Microlensing magnification A(t)')
plt.xlabel('time [arb]'); plt.ylabel('A')
plt.tight_layout(); plt.savefig(NB_OUT/'magnification_curve.png', dpi=150); plt.close()
print('Saved magnification_curve.png')

## 3) From time to effective spectrum
In an ideal ratio measurement, pure lensing would cancel out in the transit depth. In practice, **time‑varying magnification** during the observation coupled with **wavelength‑dependent exposure/weighting** (due to throughput, flagging, or rolling shutter timing) can produce wavelength‑dependent biases.

We model this by defining per‑wavelength exposure weights `W(λ)` (normalized) and computing a weighted average across time windows that intersect the lensing peak. This toy model demonstrates a possible **effective** bias.

In [ ]:
wl = base['wavelength_index'].to_numpy()
mu = base['mu'].to_numpy(float)
W = wl / wl.max()  # simple monotonic weight vs wavelength (proxy for throughput)
W = (W - W.min())/(W.ptp() + 1e-12)
W = 0.5 + 0.5*W  # scale to [0.5,1.0]

# Effective lensed spectrum: weight time samples by A(t) and wavelength weights W(λ)
# Simplified: pretend each λ samples a slightly shifted time slice (e.g., readout order)
phase_per_wl = np.linspace(-0.15, 0.15, len(wl))  # mock readout time offset per wavelength
A_per_wl = np.interp(phase_per_wl, t, A)

mu_lensed = mu * (1.0 + (A_per_wl - 1.0)* (W))  # multiplicative bias modulated by weights
residual = mu_lensed - mu

fig, ax = plt.subplots(2,1, figsize=(10,6), sharex=True)
ax[0].plot(wl, mu, lw=1.4, label='base μ')
ax[0].plot(wl, mu_lensed, lw=1.2, label='effective μ (with lensing+weights)')
ax[0].set_ylabel('μ (arb)'); ax[0].legend()
ax[1].plot(wl, residual, lw=1.2, color='tab:red', label='residual (lensed − base)')
ax[1].axhline(0, color='k', lw=0.7)
ax[1].set_xlabel('wavelength index'); ax[1].set_ylabel('Δμ (arb)'); ax[1].legend()
fig.suptitle('Effective spectrum under microlensing + wavelength weights')
fig.tight_layout(); fig.savefig(NB_OUT/'effective_spectrum_lensing.png', dpi=150); plt.close(fig)
print('Saved effective_spectrum_lensing.png')

### Symbolic band overlays (educational)
We overlay two nominal water‑band index ranges to show whether apparent residuals cluster in key regions. Replace with instrument‑accurate bands for your grid.

In [ ]:
SYM_BANDS = {
    'H2O_1': (120, 150),
    'H2O_2': (180, 220)
}

plt.figure(figsize=(10,3))
plt.plot(wl, residual, lw=1.2, color='tab:red')
for name,(a,b) in SYM_BANDS.items():
    a,b = max(0,a), min(len(wl),b)
    if b>a:
        plt.axvspan(wl[a], wl[b-1], color='gray', alpha=0.18)
        plt.text((wl[a]+wl[b-1])/2, residual.max()*0.9, name, ha='center', va='top', fontsize=8, alpha=0.8)
plt.axhline(0, color='k', lw=0.7)
plt.title('Residuals with symbolic band overlays')
plt.xlabel('wavelength index'); plt.ylabel('Δμ (arb)')
plt.tight_layout(); plt.savefig(NB_OUT/'residuals_with_bands.png', dpi=150); plt.close()
print('Saved residuals_with_bands.png')

## 4) Einstein‑ring radius sketch (quick look)
We plot the classic point‑lens image separation scale as context (qualitative sketch; not used directly in spectra).

In [ ]:
theta = np.linspace(0, 2*np.pi, 400)
x = np.cos(theta)
y = np.sin(theta)
plt.figure(figsize=(4,4))
plt.plot(x, y, lw=1.5)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Einstein ring (unit radius sketch)')
plt.axis('off')
plt.tight_layout(); plt.savefig(NB_OUT/'einstein_ring_sketch.png', dpi=150); plt.close()
print('Saved einstein_ring_sketch.png')

## 5) Export bundle
We save a compact JSON bundle and CSV for dashboards/teaching.

In [ ]:
bundle = {
  'base_source': (str(CANDS[-1].relative_to(ROOT)) if 'CANDS' in globals() and CANDS else 'synthetic'),
  'n_exp': int(len(np.linspace(-0.8,0.8,200))),
  'figures': [
      'base_spectrum.png',
      'magnification_curve.png',
      'effective_spectrum_lensing.png',
      'residuals_with_bands.png',
      'einstein_ring_sketch.png'
  ],
  'notes': 'Toy model: achromatic microlensing + wavelength/time weighting may yield effective spectral bias.'
}
(NB_OUT/'lensing_demo_bundle.json').write_text(json.dumps(bundle, indent=2))
pd.DataFrame({'wavelength_index': wl, 'mu_base': mu, 'mu_lensed': mu_lensed, 'residual': residual}).to_csv(NB_OUT/'lensing_demo_detail.csv', index=False)
print('Wrote bundle & detail CSV to', NB_OUT)

## 6) (Optional) DVC add
Register outputs for full reproducibility if your project uses DVC.

In [ ]:
import shutil, subprocess
if shutil.which('dvc'):
    try:
        subprocess.run(['dvc','add', str(NB_OUT)], check=False)
        subprocess.run(['git','add', f'{NB_OUT}.dvc', '.gitignore'], check=False)
        subprocess.run(['dvc','status'], check=False)
        print('DVC add done (non‑blocking).')
    except Exception as e:
        print('DVC step failed (non‑blocking):', e)
else:
    print('DVC not found; skipping.')

---
### Notes & caveats
- **Achromatic** lensing by itself does *not* change intrinsic spectral features; apparent biases here arise from simplified time/weight coupling used for demonstration.
- For realistic use, replace the weight model `W(λ)` and timing offsets with instrument‑specific readout/throughput and sampling.
- Keep production spectra in the CLI calibration/prediction pipeline; use this notebook for physics intuition and QA visualization only.